Volvemos a cargar el dataset y una serie de otras cosas. 

Basado en este excelente [tutorial](https://keras.io/examples/graph/gnn_citations/).

In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

citations = pd.read_csv("cora/cora.cites",
    sep="\t",
    header=None,
    names=["target", "source"],
)

column_names = ["paper_id"] + [f"term_{idx}" for idx in range(1433)] + ["subject"]
papers = pd.read_csv("cora/cora.content", sep="\t", header=None, names=column_names,
)


In [12]:
class_values = sorted(papers["subject"].unique())
class_idx = {name: id for id, name in enumerate(class_values)}
paper_idx = {name: idx for idx, name in enumerate(sorted(papers["paper_id"].unique()))}

papers["paper_id"] = papers["paper_id"].apply(lambda name: paper_idx[name])
citations["source"] = citations["source"].apply(lambda name: paper_idx[name])
citations["target"] = citations["target"].apply(lambda name: paper_idx[name])
papers["subject"] = papers["subject"].apply(lambda value: class_idx[value])

train_data, test_data = [], []

for _, group_data in papers.groupby("subject"):
    # Select around 50% of the dataset for training.
    random_selection = np.random.rand(len(group_data.index)) <= 0.5
    train_data.append(group_data[random_selection])
    test_data.append(group_data[~random_selection])

train_data = pd.concat(train_data).sample(frac=1)
test_data = pd.concat(test_data).sample(frac=1)

print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)



Train data shape: (1362, 1435)
Test data shape: (1346, 1435)


In [13]:
train_data

,paper_id,term_0,term_1,term_2,term_3,term_4,term_5,term_6,term_7,term_8,...,term_1424,term_1425,term_1426,term_1427,term_1428,term_1429,term_1430,term_1431,term_1432,subject
1272,1378,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2486,51,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
367,667,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
668,1274,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,2
414,196,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950,1466,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,3
1643,135,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
222,2647,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1231,1582,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [19]:
def create_MLP(capas_internas, dropout_rate, name=None):
    capas = []

    for capa_interna in capas_internas:
        capas.append(layers.BatchNormalization())
        capas.append(layers.Dropout(dropout_rate))
        capas.append(layers.Dense(capa_interna, activation=tf.nn.gelu))

    return keras.Sequential(capas, name=name)


## Manejo de datos específico para nuestras GNNs. 

Lo primero es que ahora las GNNs van a funcionar en base a las conexiones entro los papers (además de los features obviamente). La GNN se compila con la info del grado, por lo que el x_train y x_test solo deben tener los id de los nodos relevantes. 

In [14]:
feature_names = list(set(papers.columns) - {"paper_id", "subject"})
num_features = len(feature_names)
num_classes = len(class_idx)

# Create train and test features as a numpy array.
x_train = train_data["paper_id"].to_numpy()
x_test = test_data["paper_id"].to_numpy()
# Create train and test targets as a numpy array.
y_train = train_data["subject"]
y_test = test_data["subject"]


El segundo paso es crear una matriz de adyacencia en formato numpy, que es lo que vamos a necesitar para pasarselo a tensorflow. Por razones de formato, es mejor usar una representación esparsa, en forma de lista de pares. 

In [15]:
#Matriz en forma de lista de pares
edges = citations[["source", "target"]].to_numpy().T

#Codigo para agregar peso a cada arista, por ahora son puros 1s, todas valen lo mismo. 
edge_weights = tf.ones(shape=edges.shape[1])

# Crear (en formato tensowrflow) los features para cada nodo.
node_features = tf.cast(
    papers.sort_values("paper_id")[feature_names].to_numpy(), dtype=tf.dtypes.float32
)

#### juan: esto se puede simplificar
# el grafo es la union de estas tres cosas
graph_info = (node_features, edges, edge_weights)

print("Edges shape:", edges.shape)
print("Nodes shape:", node_features.shape)


### Esto es muy importante. 
### El primer vector es la lista de los indices de los nodos source de edges, 
### El segundo vector es la lista de los indices de los nodos target

node_indices, neighbour_indices = edges[0], edges[1]


Edges shape: (2, 5429)
Nodes shape: (2708, 1433)


### Un modelo para una capa de la GNN

Esta es la capa que va a hacer los pasos de agregación y update. 
Lamentablemente, la estructura de tensorflow nos obliga a definir estas operaciones, bastante complejas, como otras layers, por lo que procedemos a extender la clase *Layer*. Lo bueno es que funciona prácticamente igual que un *Model*.  

In [16]:
class GNNlayer(layers.Layer):
    def __init__(
        self,
        capas_internas = [32,32],
        dropout_rate=0.2,
        normalize=False,
        *args,
        **kwargs,
    ):
        super(GNNlayer, self).__init__(*args, **kwargs)

        #Hay dos redes neuronales involucradas en una capa de GNN: la primera activación de los mensajes, 
        #    y el manejo del update. 
            
        self.preprocesador = create_MLP(capas_internas, dropout_rate)

        self.updater = create_MLP(capas_internas, dropout_rate)

        
    def prepare(self, node_repesentations, weights=None):
        
#        Esta funcion pasa los mensajes por una red neuronal simple, y aplica los pesos (si hay)
        
        messages = self.preprocesador(node_repesentations)
        messages = messages * tf.expand_dims(weights, -1)
        return messages

    def aggregate(self, node_indices, neighbour_messages, node_repesentations):
        
        # Esta funcion agrega los mensajes de cada nodo, en forma de suma. 
        # recibo un vector node_indices, que es de largo [num_edges] y me dice los nodos origen de cada arista
        # matriz neighbour_messages es de forma [num_edges, (neuronas_internas)], osea [num_edges, 32] en este codigo
        # esta matriz tiene el mensaje de cada nodo que participa en la arista como nodo destino
        # la matriz node_repesentations es de la forma [num_nodes, representation_dim], contiene información de 
        # los nodos del grafo. 
        
        num_nodes = node_repesentations.shape[0]
        
                
        #### La funcion unsorted_segment_sum me suma los mensajes de todos los indices iguales en node_indices y 
        #### los deja en el i-esimo lugar; con eso sumamos los ids. Si un nodo no tiene vecinos recibe un 0
        
        aggregated_message = tf.math.unsorted_segment_sum(
            neighbour_messages, node_indices, num_segments=num_nodes
            
        )
    
        return aggregated_message

    def update(self, node_repesentations, aggregated_messages):
        
        # Para combinar los mensajes con los features de cada nodo, concatenamos. 
        # Notar que a este punto tanto node_repesentations como aggregated_messages tienen forma 
        # [num_nodes, representation_dim]. Concat me los concatena. 

        h = tf.concat([node_repesentations, aggregated_messages], axis=1)
        
        # Y aplicamos unas capas no-lineales
        
        node_embeddings = self.updater(h)

        return node_embeddings

    def call(self, inputs):
        ## Procesa los inputs para crear los embeddings. Siempre tenemos información de todo el grafo, 
        ## y operamos sobre todos los nodos en node_representations

        node_repesentations, edges, edge_weights = inputs
        
        node_indices, neighbour_indices = edges[0], edges[1]
        
        # Lo primero es una lista de vectores en donde tomo cada id en neighbour_indices 
        # y lo reemplazo por la representación de ese id. 
        # El resultado es una lista que contiene, para cada arista, la representación del target de esa arista. 
        neighbour_repesentations = tf.gather(node_repesentations, neighbour_indices)

        # Procesamos estos mensajes (posiblemente incluyendo pesos en aristas)
        neighbour_messages = self.prepare(neighbour_repesentations, edge_weights)
        
        #Los agregamos
        aggregated_messages = self.aggregate(
            node_indices, neighbour_messages, node_repesentations
        )
        
        # Y finalmente, el update. 
        return self.update(node_repesentations, aggregated_messages)


## Juntando todo en un clasificador

Ahora si, definimos un modelo igual que la vez anterior, solo que ahora usa nuestra capa! 

In [20]:
class GNNbasica(tf.keras.Model):
    def __init__(
        self,
        graph_info,
        num_classes,
        capas_internas = [32,32],
        dropout_rate=0.2,
        normalize=True,
        *args,
        **kwargs,
    ):
        
        super(GNNbasica, self).__init__(*args, **kwargs)

#LA GNN maneja información de todo el grafo, independiente del batch que procese. 

        node_features, edges, edge_weights = graph_info
        self.node_features = node_features
        self.edges = edges
        self.edge_weights = edge_weights
        
        #normalizar
        self.edge_weights = self.edge_weights / tf.math.reduce_sum(self.edge_weights)

#Las layers básicas: una capa para preprocesar todo        
        self.preprocesar = create_MLP(capas_internas, dropout_rate, name="preprocesado")
    
# dos capas de paso de mensjaes 

        self.capa1 = GNNlayer(
            capas_internas,
            dropout_rate,
            name="capa1",
        )
        # Create the second GraphConv layer.
        self.capa2 = GNNlayer(
            capas_internas,
            dropout_rate,
            name="capa2",
        )

        self.capaMA = GNNlayer(
            capas_internas,
            dropout_rate,
            name="capaMA",
        )
        
        # Create a postprocess layer.
        self.postprocess = create_MLP(capas_internas, dropout_rate, name="postprocess")
        # Create a compute logits layer.
        self.clas = layers.Dense(units=num_classes, name="logits")

        
    def call(self, batch_indices):
        
        #### Capa de preprocesado de features, para bajar la dimensionalidad
        
        nodos_preprocesados = self.preprocesar(self.node_features)
        
        #### Estos nodos preprocesados pasan por la capa1, la que agrega los mensajes de sus vecinos. 
            
        paso_mens1 = self.capa1((nodos_preprocesados,self.edges, self.edge_weights))
        
        skip1 = nodos_preprocesados + paso_mens1 
        
        paso_mens2 = self.capa2((skip1,self.edges, self.edge_weights))
        
        skip2 = paso_mens2 + skip1

        MA = self.capaMA(skip2)

        ##### Postprocesado y llegar a la categoría del nodo
        postprocesado = self.postprocess(MA)

        ##### Volvemos a poner los embeddings en el orden que demandaba segun el batch
        node_embeddings = tf.gather(postprocesado, batch_indices)

        # Readout para llegar a las categorias
        
        return self.clas(node_embeddings)
        

In [18]:

GNN = GNNbasica(
    graph_info=graph_info,
    num_classes=7,
    capas_internas=[32,32],
    dropout_rate=0.2,
    name="gnn_model",
)

#print(GNN([1, 10, 100]))

GNN.summary()





TypeError: 'int' object is not subscriptable

In [18]:
GNN.compile(
        optimizer=keras.optimizers.Adam(0.01),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
    )

# Create an early stopping callback.
early_stopping = keras.callbacks.EarlyStopping(
        monitor="val_acc", patience=50, restore_best_weights=True
    )

# A la GNN hay que darlos los ids de los nodos 


    # Fit the model.
history = GNN.fit(
        x=x_train,
        y=y_train,
        epochs=300,
        batch_size=256,
        validation_split=0.15,
        callbacks=[early_stopping],
    )



Epoch 1/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 81ms/step - acc: 0.2394 - loss: 1.9498 - val_acc: 0.2330 - val_loss: 1.8574
Epoch 2/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - acc: 0.4006 - loss: 1.7877 - val_acc: 0.4078 - val_loss: 1.6851
Epoch 3/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - acc: 0.6111 - loss: 1.4139 - val_acc: 0.4709 - val_loss: 1.5596
Epoch 4/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - acc: 0.7341 - loss: 0.7789 - val_acc: 0.5146 - val_loss: 1.9149
Epoch 5/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - acc: 0.7868 - loss: 0.6284 - val_acc: 0.6068 - val_loss: 1.3117
Epoch 6/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - acc: 0.8623 - loss: 0.3690 - val_acc: 0.6505 - val_loss: 1.1667
Epoch 7/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - acc: 0.9195 - loss: 0.2499 - val_acc: 0.6748 - val_loss: 1.4522
Epoch 8/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - acc: 0.9618 - loss: 0.1184 - val_acc: 0.6748 - val_loss: 1.7242
Epoch 9/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - acc: 0.9827 - loss: 

5/5 [==============================] - 0s 86ms/step - loss: 0.0316 - acc: 0.9912 - val_loss: 1.1568 - val_acc: 0.8557
Epoch 250/300
5/5 [==============================] - 0s 88ms/step - loss: 0.0266 - acc: 0.9947 - val_loss: 1.0264 - val_acc: 0.8657
Epoch 251/300
5/5 [==============================] - 0s 94ms/step - loss: 0.0175 - acc: 0.9947 - val_loss: 0.9971 - val_acc: 0.8507
Epoch 252/300
5/5 [==============================] - 0s 93ms/step - loss: 0.0294 - acc: 0.9921 - val_loss: 1.0891 - val_acc: 0.8159
Epoch 253/300
5/5 [==============================] - 0s 95ms/step - loss: 0.0229 - acc: 0.9938 - val_loss: 1.1051 - val_acc: 0.8109
Epoch 254/300
5/5 [==============================] - 0s 94ms/step - loss: 0.0272 - acc: 0.9930 - val_loss: 1.1562 - val_acc: 0.8209
Epoch 255/300
5/5 [==============================] - 0s 91ms/step - loss: 0.0187 - acc: 0.9930 - val_loss: 1.1787 - val_acc: 0.8308
Epoch 256/300
5/5 [==============================] - 0s 86ms/step - loss: 0.0246 - acc: 0.

In [19]:
x_test = test_data.paper_id.to_numpy()
GNN.evaluate(x=x_test, y=y_test)

42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - acc: 0.6944 - loss: 3.4674


[3.54608416557312, 0.6878742575645447]

Mucho mejor! 

In [20]:
GNN.summary()

Model: "gnn_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ preprocesado (Sequential)       │ (2708, 32)             │        52,804 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ capa1 (GNNlayer)                │ ?                      │         5,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ capa2 (GNNlayer)                │ ?                      │         5,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ postprocess (Sequential)        │ (2708, 32)             │         2,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ logits (Dense)                  │ (None, 7)              │           231 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 194,143 (758.38 KB)

 Trainable params: 63,481 (247.97 KB)

 Non-trainable params: 3,698 (14.45 KB)

 Optimizer params: 126,964 (495.96 KB)

# Actividades sugeridas

### Averigua sobre GRUs, y lee el paper de Gated Graph Sequence Neural Networks. Implementa un clasificador de acuerdo con esa tecnología. Puedes tambien ver una implementación [aquí](https://keras.io/examples/graph/gnn_citations/).  

### Averigua sobre atención en grafos, y lee el paper de Graph Attention Networks. Implementa un clasificador de acuerdo con esa tecnología. Puedes tambien ver una implementación [aquí](https://keras.io/examples/graph/gat_node_classification/).  